<a href="https://colab.research.google.com/github/Chan0226/CNN/blob/main/%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# cub200-2011
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1hbzc_P1FuxMkcabkgn9ZKinBwW683j45' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1hbzc_P1FuxMkcabkgn9ZKinBwW683j45" -O CUB_200_2011.tgz && rm -rf /tmp/cookies.txt

--2021-11-19 10:16:45--  https://docs.google.com/uc?export=download&confirm=V422&id=1hbzc_P1FuxMkcabkgn9ZKinBwW683j45
Resolving docs.google.com (docs.google.com)... 108.177.121.138, 108.177.121.102, 108.177.121.139, ...
Connecting to docs.google.com (docs.google.com)|108.177.121.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-50-docs.googleusercontent.com/docs/securesc/bejl6ccd5979qrvqjtc1tseil8fdbq2f/5ga7tdktsa34mmafnblg6h3esmuamghi/1637316975000/15424859768005087218/05533706151004306672Z/1hbzc_P1FuxMkcabkgn9ZKinBwW683j45?e=download [following]
--2021-11-19 10:16:45--  https://doc-14-50-docs.googleusercontent.com/docs/securesc/bejl6ccd5979qrvqjtc1tseil8fdbq2f/5ga7tdktsa34mmafnblg6h3esmuamghi/1637316975000/15424859768005087218/05533706151004306672Z/1hbzc_P1FuxMkcabkgn9ZKinBwW683j45?e=download
Resolving doc-14-50-docs.googleusercontent.com (doc-14-50-docs.googleusercontent.com)... 64.233.182.132, 2607:f8b0:4001:c0a::84
Connec

In [ ]:
# 압축을 풀어줘야 함
!tar -xvzf /content/CUB_200_2011.tgz

In [ ]:
base_folder = '/content/CUB_200_2011/'
image_txt_path =  base_folder + 'images.txt'
train_test_split_path = base_folder + 'train_test_split.txt'
images_path = base_folder + 'images/'

In [ ]:
from __future__ import absolute_import
from __future__ import print_function
import matplotlib.pyplot as plt
import os
import shutil
from shutil import copyfile

In [ ]:
test_folder = '/content/CUB_200_2011/test/'
train_folder = '/content/CUB_200_2011/train/'

In [ ]:
def ignore_files(dir,files):
  return [f for f in files if os.path.isfile(os.path.join(dir,f))]

In [ ]:
shutil.copytree(images_path,test_folder,ignore=ignore_files)
shutil.copytree(images_path,train_folder,ignore=ignore_files)

'/content/CUB_200_2011/train/'

In [ ]:
with open(image_txt_path) as f:
  images_lines = f.readlines()

In [ ]:
with open(train_test_split_path) as f:
  split_lines = f.readlines()

In [ ]:
test_images, train_images = 0,0

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.preprocessing import image
import os


In [ ]:
train_folder='/content/CUB_200_2011/train'
test_folder='/content/CUB_200_2011/test'


In [ ]:
class_reduce=0.1 # 부류 수 줄여서 데이터양 줄임(속도와 메모리 효율을 위해)
no_class=int(len(os.listdir(train_folder))*class_reduce) # 부류 개수


In [ ]:
x_train,y_train=[],[]
for i,class_name in enumerate(os.listdir(train_folder)):
    if i<no_class: # 13~14행이 지정한 부류만 사용
        for fname in os.listdir(train_folder+'/'+class_name):
            img=image.load_img(train_folder+'/'+class_name+'/'+fname,target_size=(224,224))
            if len(img.getbands())!=3:
                print("주의: 유효하지 않은 영상 발생",class_name,fname)
                continue
            x=image.img_to_array(img)
            x=preprocess_input(x)
            x_train.append(x)
            y_train.append(i)

In [ ]:
x_test,y_test=[],[]
for i,class_name in enumerate(os.listdir(test_folder)):
    if i<no_class: # 13~14행이 지정한 부류만 사용
        for fname in os.listdir(test_folder+'/'+class_name):
            img=image.load_img(test_folder+'/'+class_name+'/'+fname,target_size=(224,224))
            if len(img.getbands())!=3:
                print("주의: 유효하지 않은 영상 발생",class_name,fname)
                continue
            x=image.img_to_array(img)
            x=preprocess_input(x)
            x_test.append(x)
            y_test.append(i)

In [ ]:
x_train=np.asarray(x_train)
y_train=np.asarray(y_train)
x_test=np.asarray(x_test)
y_test=np.asarray(y_test)
y_train=tf.keras.utils.to_categorical(y_train,no_class)
y_test=tf.keras.utils.to_categorical(y_test,no_class)

In [ ]:
base_model=ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
cnn=Sequential()
cnn.add(base_model)
cnn.add(Flatten())
cnn.add(Dense(1024,activation='relu'))
cnn.add(Dense(no_class,activation='softmax'))

In [ ]:
cnn.compile(loss='categorical_crossentropy',optimizer=Adam(0.00002),metrics=['accuracy'])
hist=cnn.fit(x_train,y_train,batch_size=16,epochs=10,validation_data=(x_test,y_test),verbose=2)

Epoch 1/10
38/38 - 61s - loss: 2.4931 - accuracy: 0.3920 - val_loss: 1.1420 - val_accuracy: 0.6742 - 61s/epoch - 2s/step
Epoch 2/10
38/38 - 20s - loss: 0.0552 - accuracy: 0.9849 - val_loss: 0.9313 - val_accuracy: 0.7539 - 20s/epoch - 534ms/step
Epoch 3/10
38/38 - 20s - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.8877 - val_accuracy: 0.7487 - 20s/epoch - 534ms/step
Epoch 4/10
38/38 - 19s - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.8405 - val_accuracy: 0.7539 - 19s/epoch - 512ms/step
Epoch 5/10
38/38 - 19s - loss: 6.2776e-04 - accuracy: 1.0000 - val_loss: 0.8393 - val_accuracy: 0.7626 - 19s/epoch - 513ms/step
Epoch 6/10
38/38 - 19s - loss: 4.5752e-04 - accuracy: 1.0000 - val_loss: 0.8453 - val_accuracy: 0.7574 - 19s/epoch - 512ms/step
Epoch 7/10
38/38 - 19s - loss: 4.9070e-04 - accuracy: 1.0000 - val_loss: 0.8495 - val_accuracy: 0.7626 - 19s/epoch - 512ms/step
Epoch 8/10
38/38 - 19s - loss: 2.4192e-04 - accuracy: 1.0000 - val_loss: 0.8517 - val_accuracy: 0.7643 - 19s/epoch - 513ms/

In [ ]:
res=cnn.evaluate(x_test,y_test,verbose=0)
print("정확률은",res[1]*100)

정확률은 75.73657035827637


In [ ]:
# 가중치 동결방식
base_model.layers
for ly in base_model.layers:
  ly.trainable  =False

In [ ]:
cnn.compile(loss='categorical_crossentropy',optimizer=Adam(0.00002),metrics=['accuracy'])
hist=cnn.fit(x_train,y_train,batch_size=16,epochs=10,validation_data=(x_test,y_test),verbose=2)
res=cnn.evaluate(x_test,y_test,verbose=0)
print("정확률은",res[1]*100)

Epoch 1/10
38/38 - 16s - loss: 0.0018 - accuracy: 0.9983 - val_loss: 1.0214 - val_accuracy: 0.7383 - 16s/epoch - 408ms/step
Epoch 2/10
38/38 - 10s - loss: 0.0026 - accuracy: 0.9983 - val_loss: 1.3306 - val_accuracy: 0.7019 - 10s/epoch - 264ms/step
Epoch 3/10
38/38 - 10s - loss: 0.0015 - accuracy: 1.0000 - val_loss: 1.8873 - val_accuracy: 0.6586 - 10s/epoch - 264ms/step
Epoch 4/10
38/38 - 10s - loss: 7.0829e-04 - accuracy: 1.0000 - val_loss: 1.8169 - val_accuracy: 0.7106 - 10s/epoch - 265ms/step
Epoch 5/10
38/38 - 10s - loss: 0.0142 - accuracy: 0.9950 - val_loss: 1.4829 - val_accuracy: 0.7192 - 10s/epoch - 264ms/step
Epoch 6/10
38/38 - 10s - loss: 0.0157 - accuracy: 0.9950 - val_loss: 1.8474 - val_accuracy: 0.6690 - 10s/epoch - 264ms/step
Epoch 7/10
38/38 - 10s - loss: 4.3914e-04 - accuracy: 1.0000 - val_loss: 1.7760 - val_accuracy: 0.6759 - 10s/epoch - 264ms/step
Epoch 8/10
38/38 - 10s - loss: 5.9419e-05 - accuracy: 1.0000 - val_loss: 1.6889 - val_accuracy: 0.6967 - 10s/epoch - 263ms/s